In [0]:
dbutils.fs.put('/jana/sample.json',"""[{
   "data": [
      {
         "id": "X999_Y999",
         "from": {
            "name": "Tom Brady", "id": "X12"
         },
         "message": "Looking forward to 2010!",
         "actions": [
            {
               "name": "Comment",
               "link": "http://www.facebook.com/X999/posts/Y999"
            },
            {
               "name": "Like",
               "link": "http://www.facebook.com/X999/posts/Y999"
            }
         ],
         "type": "status",
         "created_time": "2010-08-02T21:27:44+0000",
         "updated_time": "2010-08-02T21:27:44+0000"
      },
      {
         "id": "X998_Y998",
         "from": {
            "name": "Peyton Manning", "id": "X18"
         },
         "message": "Where's my contract?",
         "actions": [
            {
               "name": "Comment",
               "link": "http://www.facebook.com/X998/posts/Y998"
            },
            {
               "name": "Like",
               "link": "http://www.facebook.com/X998/posts/Y998"
            }
         ],
         "type": "status",
         "created_time": "2010-08-02T21:27:44+0000",
         "updated_time": "2010-08-02T21:27:44+0000"
      }
   ]
}]""",True)

Wrote 1197 bytes.
Out[1]: True

In [0]:
from pyspark.sql.functions import explode,col

In [0]:
df=spark.read.json('/jana/sample.json',multiLine=True)
display(df)

data
"List(List(List(List(http://www.facebook.com/X999/posts/Y999, Comment), List(http://www.facebook.com/X999/posts/Y999, Like)), 2010-08-02T21:27:44+0000, List(X12, Tom Brady), X999_Y999, Looking forward to 2010!, status, 2010-08-02T21:27:44+0000), List(List(List(http://www.facebook.com/X998/posts/Y998, Comment), List(http://www.facebook.com/X998/posts/Y998, Like)), 2010-08-02T21:27:44+0000, List(X18, Peyton Manning), X998_Y998, Where's my contract?, status, 2010-08-02T21:27:44+0000))"


In [0]:
df=df.withColumn('data_ex',explode('data')).drop('data')
df.printSchema()

root
 |-- data_ex: struct (nullable = true)
 |    |-- actions: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- link: string (nullable = true)
 |    |    |    |-- name: string (nullable = true)
 |    |-- created_time: string (nullable = true)
 |    |-- from: struct (nullable = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- name: string (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-- message: string (nullable = true)
 |    |-- type: string (nullable = true)
 |    |-- updated_time: string (nullable = true)



In [0]:
df_final=df.select('data_ex.*') \
           .withColumn('actions_ex',explode('actions')).drop('actions') \
           .withColumn('actions_link',col('actions_ex.link')) \
           .withColumn('actions_name',col('actions_ex.name')) \
           .drop('actions_ex') \
           .withColumn('from_id',col('from.id')) \
           .withColumn('from_name',col('from.name')) \
           .drop('from')

In [0]:
df_final.printSchema()

root
 |-- created_time: string (nullable = true)
 |-- id: string (nullable = true)
 |-- message: string (nullable = true)
 |-- type: string (nullable = true)
 |-- updated_time: string (nullable = true)
 |-- actions_link: string (nullable = true)
 |-- actions_name: string (nullable = true)
 |-- from_id: string (nullable = true)
 |-- from_name: string (nullable = true)



In [0]:
display(df_final)

created_time,id,message,type,updated_time,actions_link,actions_name,from_id,from_name
2010-08-02T21:27:44+0000,X999_Y999,Looking forward to 2010!,status,2010-08-02T21:27:44+0000,http://www.facebook.com/X999/posts/Y999,Comment,X12,Tom Brady
2010-08-02T21:27:44+0000,X999_Y999,Looking forward to 2010!,status,2010-08-02T21:27:44+0000,http://www.facebook.com/X999/posts/Y999,Like,X12,Tom Brady
2010-08-02T21:27:44+0000,X998_Y998,Where's my contract?,status,2010-08-02T21:27:44+0000,http://www.facebook.com/X998/posts/Y998,Comment,X18,Peyton Manning
2010-08-02T21:27:44+0000,X998_Y998,Where's my contract?,status,2010-08-02T21:27:44+0000,http://www.facebook.com/X998/posts/Y998,Like,X18,Peyton Manning


In [0]:
%sql 
create database customers_db

In [0]:
df_final.write.format('csv').saveAsTable('customers_db.bala')